# Create a quick map of your region 
Region are outlined in regions_bbox_canada.json
Add any new regions of your choosing there and give a descriptive name to call and use.
Currently uses plotly to create maps
So will not trim the dataset to your region but will zimply zoom to that region. 

No interpolation is done (as in wind_comparison.ipynb) this is only raw CaSR data.

Liam.Buchart@nrcan-rncan.gc.ca
October 29, 2025

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
import json
import ipywidgets as widgets

from pathlib import Path
from IPython.display import display

In [ ]:
# slider to pick your month, dropdown to select region from the json file (just by name)
# load the json
with open("regions_bbox_canada.json", 'r') as f:
    region_data = json.load(f)
region_names = [region["name"] for region in region_data["regions"]]

region_dropdown = widgets.Dropdown(
    options=region_names,
    description='Region: ',
    style={'description_width': 'initial'}
)

Averaging_Select = widgets.Dropdown(
    options=["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "DJF", "MAM", "JJA", "SON"], 
    value="8",
    description='Period: ',
    style={'description_width': 'initial'}
    )

Variable_Select = widgets.Dropdown(
    options=["Wind Speed", "Wind Direction"], 
    value="Wind Speed",
    description='Variable: ',
    style={'description_width': 'initial'}
    )

display(Variable_Select)
display(Averaging_Select)
display(region_dropdown)

In [ ]:
selected_region = region_dropdown.value
bbox = next(region["bbox"] for region in region_data["regions"] if region["name"] == selected_region)
print(bbox)

In [ ]:
# open the dataset print variables
cwd = Path().resolve()
parent_dir = cwd.parent

if str(Averaging_Select.value).isdigit():
    # averaging over a month
    data_dir = parent_dir / "climatology" / "monthly" 
    period = f"m{int(Averaging_Select.value):02d}"
    
else:
    # seasonal average
    data_dir = parent_dir / "climatology" / "seasonal" 
    period = Averaging_Select.value

if Variable_Select.value == "Wind Direction":
    dfile = f"1990-2020_monthly_winddirection_stats_{period}.nc"
elif Variable_Select.value == "Wind Speed":
    dfile = f"1990-2020_monthly_windspeed_stats_{period}.nc"
full_path = data_dir / dfile

if full_path.exists():
    ds = xr.open_dataset(str(full_path), engine="netcdf4")
else:
    print("File does not exist.")


In [ ]:
# get the columns and create a dropwdown to select which variable you want to plot
plot_vars = list(ds.data_vars)

Plot_Select = widgets.Dropdown(
    options=plot_vars, 
    value="mean",
    description='Varaible to plot:',
    style={'description_width': 'initial'}
    )

display(Plot_Select)

In [ ]:
plot_da = ds[Plot_Select.value]  # DataArray containing mean values

center_lat = (bbox['north'] + bbox['south']) / 2
center_lon = (bbox['east'] + bbox['west']) / 2

vmin = 0   # minimum value for colorbar
vmax = 60  # maximum value for colorbar

# Flatten arrays for plotting
lats = plot_da['lat'].values.flatten()
lons = plot_da['lon'].values.flatten()
values = plot_da.values.flatten()

ptitle = f"{period} {Plot_Select.value} {Variable_Select.value}"
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    lat=lats,
    lon=lons,
    text=values,
    marker=dict(
        color=values,
        colorscale="Jet",
        colorbar=dict(title=Variable_Select.value),
        size=4,
        opacity=0.7,
    ),
    mode="markers",
    name=Variable_Select.value
))

fig.update_geos(
    visible=False, resolution=50, scope="north america",
    showcountries=True, countrycolor="Black",
    showsubunits=True, subunitcolor="Black"
 )
fig.update_geos(lataxis_showgrid=True, lonaxis_showgrid=True)

# Put title and layout settings here so title appears above shading and map elements
fig.update_layout(
    title={
        'text': ptitle,
        'x': 0.5,
        'xanchor': 'center',
        'y': 0.95,
        'yanchor': 'top'
    },
    height=800,
    margin={"r":0, "t":60, "l":0, "b":0}
 )

fig.show()

In [ ]:
title = str(period) + " 90th Percentile Wind Speed"
fig.update_geos( 
    px.scatter_map(
    lat=lats,
    lon=lons,
    color=values,
    color_continuous_scale="jet",
    range_color=[vmin, vmax],
    title=title,
    center={"lat": center_lat, "lon": center_lon},
    zoom=3)  # Adjust zoom as needed
)

fig.show()